In [1]:
import pandas as pd
import xmlrpc.client
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)


# TIEMPOS ESTANDAR

In [2]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name','workcenter_id']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['centro_produccion'] = test['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)


test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)



pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")

# Merge claficacion

test = test.rename(columns={"total_nbr_minimo":"tiempo_estimado_estandard" , "display_name":"maquina_esperada"})

pareto_prod = pareto_prod[["nom_produccion","Clasificacion"]] #Columna realmente requeridas
te_clas = test.merge(pareto_prod, how="left",on="nom_produccion") 

# Descartar productos sin ventas ya que no tienen clasificacion
sin_ventas = te_clas[te_clas["Clasificacion"].isna()] 
te_clas = te_clas[~te_clas["nom_produccion"].isin(sin_ventas["nom_produccion"])]

te_clas = te_clas[["nom_produccion","producto","Clasificacion","centro_produccion","tiempo_estimado_estandard","maquina_esperada"]]


In [3]:
te_clas[te_clas["producto"].str.contains("DA2852A")]

,nom_produccion,producto,Clasificacion,centro_produccion,tiempo_estimado_estandard,maquina_esperada
393,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,CORTE,0.0,ENTUBADORA 3
1871,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,CORTE,0.0,ENTUBADORA 2
3352,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,PLISADO,0.0,PLISADORA
3742,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,INYECCION,0.0,HENNECKE
4268,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,EMBALAJE,0.0,BOLSA
5161,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,EMBALAJE,0.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE
5720,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,EMBALAJE,0.0,CONTROL DE CALIDAD
5942,DA2852A,DA2852A F. AIRE 1_ AGRALE,A,EMBALAJE,0.0,EMBALAJE


In [4]:
te_clas["centro_produccion"].unique()

array(['CORTE', 'EMBALAJE', 'PLISADO', 'INTERNOS GUATA', 'INYECCION'],
      dtype=object)

# INFO OP

In [5]:
import xmlrpc.client
import pandas as pd
import re

# --- Configuración de conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- 1️⃣ Órdenes de producción terminadas ---
producciones_done = models.execute_kw(
    db, uid, password,
    "mrp.production", "search_read",
    [[("state", "=", "done")], ["id", "name"]]
)
df_producciones = pd.DataFrame(producciones_done)
ids_done = df_producciones["id"].tolist()

# --- 2️⃣ Work orders asociadas ---
workorders = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line.flab", "search_read",
    [
        [("production_id", "in", ids_done)],
        [
           "workcenter_id","cantidad_fabricada","display_name","operaion","production_id",
           "product_reference" ,"delay","fecha_final"

        ]
    ]
)

df_workorders = pd.DataFrame(workorders)

df_workorders["OP"] = df_workorders["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)


df_workorders["nombre"] = df_workorders["operaion"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)

df_workorders["centro_produccion"] = df_workorders["workcenter_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)

df_workorders["maquina_real"] = df_workorders["display_name"].str.split(" - ", n=1).str[0].str.strip()

df_workorders["nom_produccion1"] = df_workorders["display_name"].str.split(" - ", n=1).str[1]

df_workorders["nom_produccion"] = df_workorders["nom_produccion1"].str.extract(r"\b(DA\d+[A-Za-z]?)\b")


df_workorders = df_workorders.drop(columns=["operaion","workcenter_id","production_id","product_reference",
                                            "nom_produccion1"])

#TARDA 2 min

In [6]:
df_workorders["tiempo_real"] = (df_workorders["delay"] / df_workorders["cantidad_fabricada"]).round(2)

df_workorders.sample(5)

,delay,display_name,fecha_final,cantidad_fabricada,id,OP,nombre,centro_produccion,maquina_real,nom_produccion,tiempo_real
46685,14931.0,"INYECCION - DA8059 FILTRO AIRE 1o VOLVO,NEW HOLLAND",2023-01-26 10:50:50,50.0,48133,MO08457,YENIFFER PAOLA BAUTISTA JIMENEZ,INYECCION,INYECCION,DA8059,298.62
94280,398.0,"PLISADORA 1 - DA4690 FILTRO AIRE 2o J.DEERE, FREIGHTLINER",2025-03-04 14:38:32,500.0,95896,MO12011,YENNY KATERINE PATAQUIVA NEUSA,PLISADO,PLISADORA 1,DA4690,0.80
64811,38492.0,INYECCION MESA INTERNOS - DA4772 FILTRO AIRE INTERNO BOBCAT,2023-11-17 15:27:22,600.0,66310,MO09815,YISETH MARGARITA JULIO PEÑARANDA,INYECCION,INYECCION MESA INTERNOS,DA4772,64.15
74692,16433.0,INYECCION - DA2902 FILTRO AIRE LISTER PETTER,2024-05-16 10:04:12,200.0,76206,MO10600,YENIFFER PAOLA BAUTISTA JIMENEZ,INYECCION,INYECCION,DA2902,82.16
35564,81976.0,PLISADORA 1 - DA4598 FILTRO AIRE- CATERPILLAR.,2022-04-05 10:14:59,550.0,36754,MO07497,DUBAN ARTURO RAMIREZ AMADO,PLISADO,PLISADORA 1,DA4598,149.05


In [7]:
df_workorders["centro_produccion"].unique()

array(['CORTE', 'PLISADO', 'INTERNOS GUATA', 'INYECCION', 'EMBALAJE'],
      dtype=object)

# UNION

In [35]:
# 1️⃣ Agrupamos los tiempos estándar por producto, centro y máquina
te_agg = (
    te_clas.groupby(["nom_produccion", "centro_produccion", "maquina_esperada"], as_index=False)
    .agg({"tiempo_estimado_estandard": "sum", "Clasificacion": "first"})
    .rename(columns={"tiempo_estimado_estandard": "tiempo_estandar_total"})
)

# 2️⃣ Primer merge: coincidencia exacta (incluye máquina)
merge_exact = pd.merge(
    df_workorders,
    te_agg,
    left_on=["nom_produccion", "centro_produccion", "maquina_real"],
    right_on=["nom_produccion", "centro_produccion", "maquina_esperada"],
    how="left"
)
merge_exact["tipo_union"] = "exacta"

# 3️⃣ Detectamos los que no encontraron match exacto
faltantes = merge_exact[merge_exact["tiempo_estandar_total"].isna()].copy()

# 4️⃣ Segundo merge: solo por producto y centro (ignora máquina)
merge_flexible = pd.merge(
    faltantes.drop(columns=["maquina_esperada", "Clasificacion", "tiempo_estandar_total", "tipo_union"]),
    te_clas.groupby(["nom_produccion", "centro_produccion"], as_index=False)
            .agg({"tiempo_estimado_estandard": "sum", "Clasificacion": "first"})
            .rename(columns={"tiempo_estimado_estandard": "tiempo_estandar_total"}),
    on=["nom_produccion", "centro_produccion"],
    how="left"
)
merge_flexible["tipo_union"] = "por_centro"

# 5️⃣ Unimos ambos resultados
df_join = pd.concat(
    [merge_exact[~merge_exact["tiempo_estandar_total"].isna()], merge_flexible],
    ignore_index=True
)

# 6️⃣ Calculamos métricas
df_join["desviacion_abs"] = df_join["tiempo_real"] - df_join["tiempo_estandar_total"]
df_join["precision_%"] = (
    (df_join["tiempo_real"] / df_join["tiempo_estandar_total"]) * 100
).replace([float("inf"), -float("inf")], 0)



In [38]:
df_aaa = df_join[df_join["Clasificacion"]=="AAA"]
df_final = df_aaa[df_aaa["fecha_final"]>"2023-01-01"]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13145 entries, 24128 to 110626
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   delay                  13145 non-null  float64
 1   display_name           13145 non-null  object 
 2   fecha_final            13145 non-null  object 
 3   cantidad_fabricada     13145 non-null  float64
 4   id                     13145 non-null  int64  
 5   OP                     13145 non-null  object 
 6   nombre                 13145 non-null  object 
 7   centro_produccion      13145 non-null  object 
 8   maquina_real           13145 non-null  object 
 9   nom_produccion         13145 non-null  object 
 10  tiempo_real            13145 non-null  float64
 11  maquina_esperada       7928 non-null   object 
 12  tiempo_estandar_total  13145 non-null  float64
 13  Clasificacion          13145 non-null  object 
 14  tipo_union             13145 non-null  object 
 15  de

In [42]:
# --- Agrupar por los campos que identifican el mismo trabajo ---
cols_agrupacion = [
    "OP", "nombre", "centro_produccion", "maquina_real", "nom_produccion",
    "tiempo_estandar_total", "Clasificacion", "tipo_union"
]

df_join_sum = (
    df_final.groupby(cols_agrupacion, as_index=False)
    .agg({
        "tiempo_real": "mean",
        "cantidad_fabricada": "mean",
        "fecha_final": "max",  # conservar última fecha
        "delay": "mean",       # promedio si te interesa
    })
)

# --- Recalcular las métricas ---
df_join_sum["desviacion_abs"] = df_join_sum["tiempo_real"] - df_join_sum["tiempo_estandar_total"]
df_join_sum["precision_%"] = (
    (df_join_sum["tiempo_real"] / df_join_sum["tiempo_estandar_total"]) * 100
).replace([float("inf"), -float("inf")], 0)


In [43]:
df_join_sum[(df_join_sum["nom_produccion"]=="DA4772A") & (df_join_sum["OP"]=="MO09887") ]

,OP,nombre,centro_produccion,maquina_real,nom_produccion,tiempo_estandar_total,Clasificacion,tipo_union,tiempo_real,cantidad_fabricada,fecha_final,delay,desviacion_abs,precision_%
3911,MO09887,ALVARO MELO PRADA,CORTE,ENTUBADORA 4,DA4772A,35.0,AAA,exacta,86.675,300.0,2023-11-16 15:43:16,26002.5,51.675,247.642857
3912,MO09887,CLARA MARLEN OSPINA CASTRO,EMBALAJE,EMBALAJE,DA4772A,25.0,AAA,exacta,0.950,300.0,2023-11-18 08:42:25,286.0,-24.050,3.800000
3913,MO09887,DANIEL CASSIS FORERO,CORTE,ENTUBADORA 4,DA4772A,35.0,AAA,exacta,8.170,300.0,2023-11-14 11:44:35,2450.0,-26.830,23.342857
3914,MO09887,ERIKA YULIEDT UMAÑA SABOGAL,EMBALAJE,MARCADO/HOTMELT/ARANDELA/EMPAQUE,DA4772A,20.0,AAA,exacta,17.790,300.0,2023-11-18 07:08:21,5338.0,-2.210,88.950000
3915,MO09887,ESTEFANNY ALVAREZ VILLANUEVA,EMBALAJE,EMBALAJE,DA4772A,25.0,AAA,exacta,11.450,300.0,2023-11-20 16:04:16,3434.0,-13.550,45.800000
3916,MO09887,FRANCY CAROLINA PINZON CHACON,INTERNOS GUATA,GUATA,DA4772A,40.0,AAA,exacta,104.670,300.0,2023-11-16 13:06:38,31400.0,64.670,261.675000
3917,MO09887,INGRID CAROLINA GOMEZ DELPRADO,EMBALAJE,CONTROL DE CALIDAD,DA4772A,30.0,AAA,exacta,20.260,300.0,2023-11-18 08:18:36,6078.0,-9.740,67.533333
3918,MO09887,KATHERIN ANDREA GIL AGUILAR,INTERNOS GUATA,GUATA,DA4772A,40.0,AAA,exacta,76.120,300.0,2023-11-17 12:44:30,22835.0,36.120,190.300000
3919,MO09887,MARIA ANGELICA HERRERA LOPEZ,INYECCION,INYECCION MESA INTERNOS,DA4772A,90.0,AAA,por_centro,88.535,300.0,2023-11-17 15:28:00,26560.5,-1.465,98.372222
3920,MO09887,MARIA HELENA DELPRADO PEREZ,EMBALAJE,CONTROL DE CALIDAD,DA4772A,30.0,AAA,exacta,11.990,300.0,2023-11-18 08:19:10,3598.0,-18.010,39.966667


In [19]:
df_final.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/tiempo_estimado_estandard.xlsx")

In [25]:
df_final["nom_produccion"].unique()

array(['DA2020', 'DA4570', 'DA2738', 'DA2666', 'DA2772', 'DA4570A',
       'DA2690', 'DA2570', 'DA4982', 'DA4690', 'DA3066', 'DA8168',
       'DA4671', 'DA2968', 'DA2957', 'DA2982', 'DA4783', 'DA4666',
       'DA4772', 'DA2671', 'DA2547', 'DA2702', 'DA2612', 'DA2783',
       'DA8108', 'DA2926', 'DA2788', 'DA2937', 'DA4772A'], dtype=object)